In [18]:
# installed globally
import faiss

In [19]:
#pip install - U sentence-transformers
# quickstart guise: https://www.sbert.net/docs/quickstart.html
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
# SENTENCE TRANSFORMER INSTRUCTION and EXAMPLE
from sentence_transformers import SentenceTransformer

# Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']

# Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")


In [11]:
# efficient vector search library
from faiss import IndexFlatL2

In [12]:
# initialize the faiss index with the dimension of the embeddings, e.g. 768, 1024, 2048
index = IndexFlatL2(1024)


In [13]:
#read the file as a single string
with open('Energy metabolism fuel selection and body weight regulation.txt') as f:
    article = f.read()    

In [14]:
# chunk the file into passages of length 2000 with overlaps of 200 characters
passages = []
for i in range(0, len(article), 1800):
    passages.append({'passage_text': article[i: i + 2000]})


In [15]:
# embed the documents as vectors
# Sammy encoder style: corpus = retriever.encode_corpus(passages)
corpus = model.encode(passages)


In [ ]:
# add the vector embeddings to the faiss index
index.add(corpus)


In [ ]:
question = "Where was Ice Spice born?"


In [ ]:
# encode the question as a vector
query = retriever.encode_queries([question])


In [ ]:
# retrieve the 3 nearest neighbors of the query vector from the document vectors
# the distances are stored in D, the indices of the documents are stored in I
D, I = index.search(query, k=3)

In [ ]:
# get the associated passage strings
relevant_documents = [passages[i] for i in I[0]]

In [ ]:
openai_api_key="sike"

In [ ]:
import openai

In [ ]:
openai.api_key = openai_api_key

In [ ]:
# chat completion takes as input a list of messages in the format below
messages = [{'role': 'system', 'content': f"""
                You are an intelligent conversational agent, who can chat with the user in a friendly manner, and also answer questions using the provided context.
                If you answer a question use in-line citations (e.g. [1]).
                """}]


def ask(question):
    messages.append({'role': 'user', 'content': question})
    query = retriever.encode_queries([question])
    D, I = index.search(query, k=3)
    relevant_documents = [passages[i] for i in I[0]]
    # specify the chat model and the messages
    response = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=messages + [
                                            {'role': 'system', 'content': f"Relevant documents: 1. {relevant_documents[0]['passage_text']} \n 2. {relevant_documents[1]['passage_text']} \n 3. {relevant_documents[2]['passage_text']} \n"}]).choices[0]['message']['content']
    messages.append({'role': 'assistant', 'content': response})
    print(response + '\n')
    for i, doc in enumerate(relevant_documents):
        print(f"[{i}]   {doc['passage_text']} \n")
    return response

In [ ]:
ask('what year was she born')

In [ ]:
messages
